In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
!pip install geopandas
import geopandas as gpd
from scipy.optimize import curve_fit
import seaborn as sns
sns.set_theme()

from math import radians, cos, sin, asin, sqrt

!pip install basemap
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

!pip install mapclassify

IMPORTATION ET PREMIERE VISUALISATION DE NOS JEUX DE DEONNEES

In [ ]:
# Importation des jeux de données :
salaire = pd.read_csv('/content/drive/MyDrive/French Industry/net_salary_per_town_categories.csv', sep = ',')
entreprise = pd.read_csv('/content/drive/MyDrive/French Industry/base_etablissement_par_tranche_effectif.csv', sep = ',')
geo= pd.read_csv('/content/drive/MyDrive/French Industry/name_geographic_information.csv', sep = ',')
population = pd.read_csv('/content/drive/MyDrive/French Industry/population.csv', sep = ',')
departements = gpd.read_file('/content/drive/MyDrive/French Industry/departements.geojson')

In [ ]:
salaire.head()

In [ ]:
salaire.info()

In [ ]:
entreprise.head()

In [ ]:
entreprise.info()

In [ ]:
geo.head()

In [ ]:
geo.info()

In [ ]:
population.head()

In [ ]:
population.info()

PREPARATION DES DONNEES 

DATASET : SALAIRE

In [ ]:
# Changement du type de la colonne CODGEO 
salaire = salaire[salaire["CODGEO"].apply(lambda x: str(x).isdigit())]
salaire["CODGEO"] = salaire["CODGEO"].astype(int)

In [ ]:
# Verification : 
salaire.info()

DATASET : ENTREPRISE

In [ ]:
# Changement du type de la colonne CODGEO 
entreprise = entreprise[entreprise["CODGEO"].apply(lambda x: str(x).isdigit())]
entreprise["CODGEO"] = entreprise["CODGEO"].astype(int)
# Changement du type de la colonne DEP :
entreprise["DEP"] = entreprise["DEP"].astype(int)

In [ ]:
# Verification : 
entreprise.info()

PREPARATION POUR VISUALISATION :

- CREATION D'UN NOUVEAU DATAFRAME A PARTIR D'ENTREPRISE : 

In [ ]:
# première partie : on regroupe les colonnes par taille :
entreprise_sep['Micro'] = entreprise['E14TS1'] + entreprise['E14TS6']
entreprise_sep['Petite'] = entreprise['E14TS10'] + entreprise['E14TS20']
entreprise_sep['Medium'] = entreprise['E14TS50'] + entreprise['E14TS100']
entreprise_sep['Large'] = entreprise['E14TS200'] + entreprise['E14TS500']
entreprise_sep['Sum'] = entreprise['E14TS1'] + entreprise['E14TS6'] + entreprise['E14TS10'] + entreprise['E14TS20'] + entreprise['E14TS50'] + entreprise['E14TS100'] + entreprise['E14TS200'] + entreprise['E14TS500']

In [ ]:
# deuxiieme partie on ajoute les pourcentage des colonnes dans de nouvelles colonnes : 
entreprise_sep['Micro%'] = entreprise_sep['Micro'] * 100 / entreprise_sep['Sum']
entreprise_sep['Petite%'] = entreprise_sep['Petite'] * 100 / entreprise_sep['Sum']
entreprise_sep['Medium%'] = entreprise_sep['Medium'] * 100 / entreprise_sep['Sum']
entreprise_sep['Large%']= entreprise_sep['Large'] * 100 / entreprise_sep['Sum']

In [ ]:
relevant_columns = ['CODGEO','LIBGEO', 'REG', 'DEP','Sum', 'Micro', 'Petite', 'Medium', 'Large','Micro%', 'Petite%', 'Medium%', 'Large%']
entreprise_sep = entreprise_sep[relevant_columns]

In [ ]:
entreprise_sep.head()

In [ ]:
entreprise_sep.head()

In [ ]:
# On remet la colonne departement en int :
entreprise_sep["DEP"] = entreprise_sep["DEP"].astype(int)

AFIN DE TRAVAILLER PLUS FACILEMENT ON REGROUPE LE DATASET GEO AVEC ENTREPRISE 

In [ ]:
# On rassemble les deux dataset geo et entreprise_sep :
entreprise_geo = entreprise_sep.merge(geo, how = 'left', left_on = 'CODGEO', right_on = 'code_insee') 

In [ ]:
entreprise_geo.head()

In [ ]:
entreprise_geo.info()

ON CREER UN NVX DATASET REGION POUR RASSEMBLER NOS INFORMATIONS PAR REGION ET RENDRE NOS VISUELS PLUS LISIBLES

In [ ]:
# On regroupe par nom de région en faisant la somme, on ne garde que la colonne avec le nombre total d'entreprise :
regions = entreprise_geo[["nom_région", "Sum"]].groupby("nom_région").sum().sort_values("Sum", ascending=False).reset_index().reset_index()

In [ ]:
regions.rename(columns={'index': 'region_rang',}, inplace=True)

In [ ]:
regions.head(20)

In [ ]:
# Instanciation d'un barplot :
plt.figure(figsize=(40, 20))
plt.title(label = 'Classement des entreprises par région')
sns.barplot(y= regions['nom_région'], x = regions['Sum'], data=regions);

ON MODIFIE LE DATASET GEO DANS LE BUT DE FAIRE UNE CARTE DE LA FRANCE :

In [ ]:
# On effectue les modification necessaire pour utiliser notre dataset : 
geo["longitude"] = geo["longitude"].apply(lambda x: str(x).replace(',','.'))

mask = geo["longitude"] == '-'
geo.drop(geo[mask].index, inplace=True)

geo.dropna(subset = ["longitude", "latitude"], inplace=True)

geo["longitude"] = geo["longitude"].astype(float)

In [ ]:
geo.drop_duplicates(subset=["code_insee"], keep="first", inplace=True)

In [ ]:
def distance(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1 = radians(lon1)
    lat1 = radians(lat1)
    lon2 = radians(lon2)
    lat2 = radians(lat2)
    
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

def distance_bigger_than(limit, lon1, lat1, lon2, lat2):
    dist = distance(lon1, lat1, lon2, lat2)
    if dist > limit:
        return 0
    else:
        return 1

In [ ]:
# Extraction des coordonnées de Paris
paris_lat = geo.loc[geo['nom_commune'] == "Paris"].iloc[0]["latitude"]
paris_lon = geo.loc[geo['nom_commune'] == "Paris"].iloc[0]["longitude"]

distances = []

# calcule des valeurs de PARIS_CLOSE
for index, row in geo.iterrows():
    distances.append(distance_bigger_than(30, row["longitude"], row["latitude"], paris_lon, paris_lat))

# ajout de la nouvelle colonne a notre dataframe
geo["PARIS_CLOSE"] = pd.Series(distances, index=geo.index)

In [ ]:
# Extraction des coordonnée des grandes villes 
majors =  geo[geo["nom_commune"] == geo["chef.lieu_région"]]
major_lats = majors["latitude"].tolist()
major_lons = majors["longitude"].tolist()

distances = []

# calcule des valeurs de MAJOR_CITY_DISTANCE
for index, row in geo.iterrows():
    
    single_distances = []
    for lat, lon in zip(major_lats, major_lons):
        single_distances.append(int(distance(row["longitude"], row["latitude"], lon, lat)))
    
    distances.append(min(single_distances))

# ajout de la nouvelle colonne a notre dataframe
geo["MAJOR_CITY_DISTANCE"] = pd.Series(distances, index=geo.index)

Creation d'un dataset avec seulement la colonne E14TST

In [ ]:
total_entreprise = entreprise[['CODGEO','LIBGEO', 'REG', 'DEP', 'E14TST']]

In [ ]:
total_entreprise.info()

In [ ]:
total_entreprise_geo = total_entreprise.merge(geo, how = 'left', left_on = 'CODGEO', right_on = 'code_insee') 

AJOUT DU DATASET DEPARTEMENTS (AFIN DE DIVISER NOTRE CARTE DE LA FRANCE EN DEPARTEMENTS)

In [ ]:
#Visualisation
departements.info()

In [ ]:
# On regroupe par département en faisant la somme 
entreprises_par_dep = total_entreprise_geo.groupby("numéro_département").sum()
# On regroupe avec le dataset departements
departements_entre = departements.merge(entreprises_par_dep, how="left", left_on="code", right_index=True)

In [ ]:
departements_entre.info()

In [ ]:
# On enleve les valeurs manquantes
departements_entre.dropna(subset = ["latitude"], inplace=True)

In [ ]:
entreprises_par_dep.head()

In [ ]:
entreprises_par_dep.reset_index(level=0, inplace=True)

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,14))
ax.set_title('Entreprises par départements', size=20, x = 0.25, y=0.90)
fig.patch.set_facecolor((202/255, 204/255, 206/255))
departements_entre.plot(ax=ax, column="E14TST", cmap=plt.cm.OrRd, scheme='fisher_jenks', k=20, legend=True)
leg = ax.get_legend()
ax.set_axis_off()
leg.set_bbox_to_anchor((0., 0., 0.2, 0.45))
leg.set_title("Nombre total d'entreprise")

VISUALISATION DU DATASET SALAIRE

CREATION D'UN NVX DATAFRAME : 

In [ ]:
# Création d'un nouveau dataframe en separant par tranche d'age et sexe 
new_age = {}
new_age["SNHMF1814"] = {"Genre": 1, "Tranche_age": '18-25'}
new_age["SNHMF2614"] = {"Genre": 1, "Tranche_age": '26-50'}
new_age["SNHMF5014"] = {"Genre": 1, "Tranche_age": '> 50'}
new_age["SNHMH1814"] = {"Genre": 2, "Tranche_age": '18-25'}
new_age["SNHMH2614"] = {"Genre": 2, "Tranche_age": '26-50'}
new_age["SNHMH5014"] = {"Genre": 2, "Tranche_age": '> 50'}

reformatted_salaire = []
for index, row in salaire.iterrows():
    for key, value in new_age.items(): 
        row_dict = {}
        row_dict["CODGEO"] = row["CODGEO"]
        row_dict["Genre"] = value["Genre"]
        row_dict["Tranche_age"] = value["Tranche_age"]
        row_dict["Salaire"] = row[key]
        reformatted_salaire.append(row_dict)
        
new_salaire_age = pd.DataFrame(reformatted_salaire) 

In [ ]:
new_salaire_age.head()

In [ ]:
# Graphique montrant les différences de salaires entre les hommes et les femmes par tranche d'âge

Salaire_F = new_salaire_age.loc[(new_salaire_age['Genre'] == 1)]
Salaire_H = new_salaire_age.loc[(new_salaire_age['Genre'] == 2)]

plt.figure(figsize=(20, 8))

plt.subplot(121)
plt.title(label = 'Salaire des femmes')
sns.barplot(x= Salaire_F['Tranche_age'], y = Salaire_F['Salaire'], data=Salaire_F);
plt.ylim([0, 20])

plt.subplot(122)
plt.title(label = 'Salaire des hommes')
sns.barplot(x= Salaire_H['Tranche_age'], y =Salaire_H ['Salaire'], data=Salaire_H);
plt.ylim([0, 20])

In [ ]:
# Création d'un nouveau dataframe en separant par sexe et poste :  
new = {}
new["SNHMFC14"] = {"Genre": 1, "Poste": 4}
new["SNHMFP14"] = {"Genre": 1, "Poste": 3}
new["SNHMFE14"] = {"Genre": 1, "Poste": 2}
new["SNHMFO14"] = {"Genre": 1, "Poste": 1}
new["SNHMHC14"] = {"Genre": 2, "Poste": 4}
new["SNHMHP14"] = {"Genre": 2, "Poste": 3}
new["SNHMHE14"] = {"Genre": 2, "Poste": 2}
new["SNHMHO14"] = {"Genre": 2, "Poste": 1}

reformatted_salaire = []
for index, row in salaire.iterrows():
    for key, value in new.items(): 
        row_dict = {}
        row_dict["CODGEO"] = row["CODGEO"]
        row_dict["Genre"] = value["Genre"]
        row_dict["Poste"] = value["Poste"]
        row_dict["Salaire"] = row[key]
        reformatted_salaire.append(row_dict)
        
new_salaire = pd.DataFrame(reformatted_salaire) 

In [ ]:
# Graphique montrant les différences de salaires entre les hommes et les femmes en fonction du poste
job_F = new_salaire.loc[(new_salaire['Genre'] == 1)]
job_H = new_salaire.loc[(new_salaire['Genre'] == 2)]

plt.figure(figsize=(20, 20))

plt.subplot(221)
plt.title(label = 'Salaire des femmes en fonction de leur poste')
sns.barplot(x= job_F['Poste'], y = job_F['Salaire'], data=job_F, label = ['Travailleurs', 'Employés', 'Cadres Moyens', 'Cadres']);
plt.ylim([0, 30])
plt.legend()

plt.subplot(222)
plt.title(label = 'Salaire des hommes en fonction de leur poste')
sns.barplot(x= job_H['Poste'], y =job_H ['Salaire'], data=job_H, label = ['Travailleurs', 'Employés', 'Cadres Moyens', 'Cadres']);
plt.ylim([0, 30])
plt.legend()

In [ ]:
plt.title(label='Différence de Salaire hommes / femmes')
sns.barplot(x = new_salaire['Genre'], y = new_salaire['Salaire'], data=new_salaire,label = ['Femmes', 'Hommes']);
plt.legend()

CREATION DE LA CARTE DE LA FRANCE CETTE FOIS CI POUR LES SALAIRES :

In [ ]:
# On assemble salaire avec geo 
salaire_location = salaire.merge(geo, how="left", left_on='CODGEO', right_on="code_insee")

In [ ]:
# On regroupe la dataframe par departement : 
salaires_par_dep = salaire_location.groupby("numéro_département").mean()
# On regroupe avec de dataset departements :
departements_salaire = departements.merge(salaires_par_dep, how="left", left_on="code", right_index=True)
# On supprime les valaeurs manquantes :
departements_salaire.dropna(subset = ["longitude"], inplace=True)

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,14))
ax.set_title('Salaires par départements', size=20, x = 0.25, y=0.90)
fig.patch.set_facecolor((202/255, 204/255, 206/255))
departements_salaire.plot(ax=ax, column="SNHM14_y", cmap=plt.cm.OrRd, scheme='fisher_jenks', k=15, legend=True)
leg = ax.get_legend()
ax.set_axis_off()
leg.set_bbox_to_anchor((0., 0., 0.2, 0.45))
leg.set_title("Moyenne net salaires")

Visualisation par rapport à la distance des grandes villes

In [ ]:
plt.figure(figsize=(16,10))
sns.set_style("whitegrid")
plt.title('Salaire en fonction de la distance avec une grande ville', fontsize=20, fontweight='bold', y=1.05,)
plt.xlabel('Distance des grandes villes proche [km]', fontsize=15)
plt.ylabel('Salaire moyen [€/heure]', fontsize=15)

years = salaire_location["MAJOR_CITY_DISTANCE"].values
memory = salaire_location["SNHM14"].values

plt.scatter(years, memory, edgecolors='black')
plt.show()

ANALYSES STATISTIQUES 

DATASET SALAIRE :

In [ ]:
print('INDICATEURS RECAPITULATIF DU DF SALARY :', '\n')
print('SALAIRE NET MOYEN Femme')
print("la valeur minimale est :", salaire['SNHMF14'].min())
print("la valeur maximale est :", salaire['SNHMF14'].max())
print("la valeur mediane est :", salaire['SNHMF14'].median(), '\n')

q1, q2, q3 = salaire['SNHMF14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Femme : CADRES')
print("la valeur minimale est :", salaire['SNHMFC14'].min())
print("la valeur maximale est :", salaire['SNHMFC14'].max())
print("la valeur mediane est :", salaire['SNHMFC14'].median(), '\n')

q1, q2, q3 = salaire['SNHMFC14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Femme : CADRES MOYEN')
print("la valeur minimale est :", salaire['SNHMFP14'].min())
print("la valeur maximale est :", salaire['SNHMFP14'].max())
print("la valeur mediane est :", salaire['SNHMFP14'].median(), '\n')

q1, q2, q3 = salaire['SNHMFP14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Femme: EMPLOYES')
print("la valeur minimale est :", salaire['SNHMFE14'].min())
print("la valeur maximale est :", salaire['SNHMFE14'].max())
print("la valeur mediane est :", salaire['SNHMFE14'].median(), '\n')

q1, q2, q3 =salaire['SNHMFE14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Femme: TRAVAILLEURS')
print("la valeur minimale est :", salaire['SNHMFO14'].min())
print("la valeur maximale est :", salaire['SNHMFO14'].max())
print("la valeur mediane est :", salaire['SNHMFO14'].median(), '\n')

q1, q2, q3 = salaire['SNHMFO14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('INDICATEURS RECAPITULATIF DU DF SALARY :', '\n')
print('SALAIRE NET MOYEN Homme')
print("la valeur minimale est :", salaire['SNHMH14'].min())
print("la valeur maximale est :", salaire['SNHMH14'].max())
print("la valeur mediane est :", salaire['SNHMH14'].median(), '\n')

q1, q2, q3 = salaire['SNHMH14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Homme: CADRES')
print("la valeur minimale est :", salaire['SNHMHC14'].min())
print("la valeur maximale est :", salaire['SNHMHC14'].max())
print("la valeur mediane est :", salaire['SNHMHC14'].median(), '\n')

q1, q2, q3 = salaire['SNHMHC14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Homme: CADRES MOYEN')
print("la valeur minimale est :", salaire['SNHMHP14'].min())
print("la valeur maximale est :", salaire['SNHMHP14'].max())
print("la valeur mediane est :", salaire['SNHMHP14'].median(), '\n')

q1, q2, q3 = salaire['SNHMHP14'].quantile(q=[0.25, 0.5, 0.75])

print('SALAIRE NET MOYEN Homme: EMPLOYE')
print("la valeur minimale est :", salaire['SNHMHE14'].min())
print("la valeur maximale est :", salaire['SNHMHE14'].max())
print("la valeur mediane est :", salaire['SNHMHE14'].median(), '\n')

q1, q2, q3 =salaire['SNHMHE14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Homme : TRAVAILLEUR')
print("la valeur minimale est :", salaire['SNHMHO14'].min())
print("la valeur maximale est :", salaire['SNHMHO14'].max())
print("la valeur mediane est :", salaire['SNHMHO14'].median(), '\n')

q1, q2, q3 = salaire['SNHMHO14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

In [ ]:
salaire.std()

In [ ]:

print('SALAIRE NET MOYEN Femme: 18-25')
print("la valeur minimale est :", salaire['SNHMF1814'].min())
print("la valeur maximale est :", salaire['SNHMF1814'].max())
print("la valeur mediane est :", salaire['SNHMF1814'].median(), '\n')

q1, q2, q3 = salaire['SNHMF1814'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Femme: 26-50 ')
print("la valeur minimale est :", salaire['SNHMF2614'].min())
print("la valeur maximale est :", salaire['SNHMF2614'].max())
print("la valeur mediane est :", salaire['SNHMF2614'].median(), '\n')

q1, q2, q3 = salaire['SNHMF2614'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Femme: 50 et +')
print("la valeur minimale est :", salaire['SNHMF5014'].min())
print("la valeur maximale est :", salaire['SNHMF5014'].max())
print("la valeur mediane est :", salaire['SNHMF5014'].median(), '\n')

q1, q2, q3 = salaire['SNHMF5014'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')


print('SALAIRE NET MOYEN Homme : 18-25')
print("la valeur minimale est :", salaire['SNHMH1814'].min())
print("la valeur maximale est :", salaire['SNHMH1814'].max())
print("la valeur mediane est :", salaire['SNHMH1814'].median(), '\n')

q1, q2, q3 = salaire['SNHMH1814'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Homme : 26-50')
print("la valeur minimale est :", salaire['SNHMH2614'].min())
print("la valeur maximale est :", salaire['SNHMH2614'].max())
print("la valeur mediane est :", salaire['SNHMH2614'].median(), '\n')

q1, q2, q3 = salaire['SNHMH2614'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN Homme: 50 et +')
print("la valeur minimale est :", salaire['SNHMH5014'].min())
print("la valeur maximale est :", salaire['SNHMH5014'].max())
print("la valeur mediane est :", salaire['SNHMH5014'].median(), '\n')

q1, q2, q3 = salaire['SNHMH5014'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

In [ ]:
print('INDICATEURS RECAPITULATIF DU DF SALARY :', '\n')
print('SALAIRE NET MOYEN')
print("la valeur minimale est :", salaire['SNHM14'].min())
print("la valeur maximale est :", salaire['SNHM14'].max())
print("la valeur mediane est :", salaire['SNHM14'].median(), '\n')

q1, q2, q3 = salaire['SNHM14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN : CADRES')
print("la valeur minimale est :", salaire['SNHMC14'].min())
print("la valeur maximale est :", salaire['SNHMC14'].max())
print("la valeur mediane est :", salaire['SNHMC14'].median(), '\n')

q1, q2, q3 = salaire['SNHMC14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN : CADRES MOYEN')
print("la valeur minimale est :", salaire['SNHMP14'].min())
print("la valeur maximale est :", salaire['SNHMP14'].max())
print("la valeur mediane est :", salaire['SNHMP14'].median(), '\n')

q1, q2, q3 = salaire['SNHMP14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN : EMPLOYES')
print("la valeur minimale est :", salaire['SNHME14'].min())
print("la valeur maximale est :", salaire['SNHME14'].max())
print("la valeur mediane est :", salaire['SNHME14'].median(), '\n')

q1, q2, q3 = salaire['SNHME14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

print('SALAIRE NET MOYEN : TRAVAILLEURS')
print("la valeur minimale est :", salaire['SNHMO14'].min())
print("la valeur maximale est :", salaire['SNHMO14'].max())
print("la valeur mediane est :", salaire['SNHMO14'].median(), '\n')

q1, q2, q3 = salaire['SNHMO14'].quantile(q=[0.25, 0.5, 0.75])

print("les quartiles sont :", 'q1 =', q1, ' / q2 = ', q2, ' / q3 = ', q3, '\n')

In [ ]:
moyenne_X2 = salaire["SNHMF14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMH14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMFC14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMHC14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMFP14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMHP14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMFE14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMHE14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMFO14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMHO14"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMF1814"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMH1814"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMF2614"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMH2614"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 =salaire["SNHMF5014"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

moyenne_X2 = salaire["SNHMH5014"].mean()
print("Moyenne calculée avec la commande python: ", moyenne_X2)
print("\n")

In [ ]:
salaire.corr()

In [ ]:
cor = salaire.corr()
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(cor, annot=True, ax=ax, cmap='crest');

print('''
On remarque une correlation forte entre les salaires net moyen par catégorie d'emploi et les 
salaires nets moyens par catégorie d'âge ''')

DATSET : ENTREPRISE

In [ ]:
entreprise_stat = entreprise[['LIBGEO','CODGEO','DEP', 'E14TST', 'E14TS1', 'E14TS6', 'E14TS10', 'E14TS20', 'E14TS50', 'E14TS100', 'E14TS200', 'E14TS500' ]]

In [ ]:
entreprise_stat.describe()

In [ ]:
entreprise.corr()

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(15,15))
plt.title('Correlation des différentes valeurs', y=1.05, size=15)
sns.heatmap(entreprise_corr.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)
plt.show()

CREATION D4UN DATAFRAME UNIQUE POUR NOTRE OBJECTIF : PREDIRE LE SALAIRE EN FRANCE

In [ ]:
french_industry = new_salaire.merge(geo, how = 'left', left_on = 'CODGEO', right_on = 'code_insee')

In [ ]:
# On renomme une colonne d'entreprise pour fusionner avec french_industry
entreprise.rename(columns = {'LIBGEO' : 'nom_commune'}, inplace = True)
# On fusionne
french_industry = french_industry.merge(right = entreprise, on = 'nom_commune', how = 'outer')

In [ ]:
french_industry.head()

In [ ]:
french_industry.info()

In [ ]:
# On a remarqué qu'il y avait des valeurs manquantes donc on les enleve :
french_industry.dropna(inplace = True)

In [ ]:
# On réduit notre dataframe pour n'avoir que les valeurs importante pour nous
french_industry = french_industry[["Poste", "Genre", "PARIS_CLOSE", "MAJOR_CITY_DISTANCE", "Micro", "Petite", "Medium", "Large", "Sum", "Salaire"]]

In [ ]:
# On verifie la correlation 
colormap = plt.cm.viridis
plt.figure(figsize=(15,15))
plt.title('Correlation des différentes valeurs', y=1.05, size=15)
sns.heatmap(french_industry.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)
plt.show()

ON PASSE A NOTRE MACHINE LEARNING

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Separation des données du dataframe feats variable explicatives et target variable cible
feats = french_industry.drop('Salaire', axis = 1)
target = french_industry['Salaire']

In [ ]:
# Création d'un ensemble d'entrainement et un ensemble test
X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.25, random_state = 42)

In [ ]:
print("Train Set:", X_train.shape)
print("Test Set:", X_test.shape)

In [ ]:
# Standardisation 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train.head()

LINEARREGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression

regressor_lr = LinearRegression()
regressor_lr.fit(X_train, y_train)

# Prediction du resultat sur test
y_pred_lr = regressor_lr.predict(X_test)

DECISIONTREEREGRESSOR

In [ ]:
from sklearn.tree import DecisionTreeRegressor

regressor_dtr = DecisionTreeRegressor(random_state = 42)
regressor_dtr.fit(X_train, y_train)

# Prediction du resultat sur test
y_pred_dtr = regressor_dtr.predict(X_test)

RANDOMFORESTREGRESSOR

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor_rfr = RandomForestRegressor(n_estimators = 10, random_state = 42)
regressor_rfr.fit(X_train, y_train)

# Prediction du resultat sur test
y_pred_rfr = regressor_rfr.predict(X_test)

RECHERCHE DE L'ACCURACY

In [ ]:
from sklearn.linear_model import LinearRegression
cl1 = LinearRegression()
cl1.fit(X_train_scaled, y_train)
print("Accuracy score de la Régression Linéaire : ",
      cl1.score(X_test_scaled, y_test))

from sklearn.tree import DecisionTreeRegressor
cl2 = DecisionTreeRegressor()
cl2.fit(X_train_scaled, y_train)
print("Accuracy score de l'Arbre de Décision de regression : ",
      cl2.score(X_test_scaled, y_test))

from sklearn.ensemble import RandomForestRegressor
cl3 = RandomForestRegressor()
cl3.fit(X_train_scaled, y_train)
print("Accuracy score du Random Forest : ",
      cl3.score(X_test_scaled, y_test))

RECHERCHE DE LA R2

In [ ]:
def standard_error(y_pred, y_test):
    sum_value = 0
    for p, t in zip(y_pred, y_test):
        sum_value += (p - t)**2
    
    return((sum_value/len(y_test))**(1/2))

In [ ]:
from sklearn.metrics import r2_score

score_lr = r2_score(y_pred_lr, y_test)
print("r2 pour la regression linéaire est égale à : " + str(round(score_lr, 3)))
score_dtr = r2_score(y_pred_dtr, y_test)
print("r2 pour l'arbre de decision regression est égal à : " + str(round(score_dtr, 3)))
score_rfr = r2_score(y_pred_rfr, y_test)
print("r2 pour la random forest regression est égal à : " + str(round(score_rfr, 3)))

In [ ]:
error_lr = standard_error(y_pred_lr.tolist(), y_test.tolist())
print("standard error pour la regression linéaire est égale à :" + str(round(error_lr, 3)))
error_dtr = standard_error(y_pred_dtr.tolist(), y_test.tolist())
print("standard error pour l'arbre de decision regression est égal à : " + str(round(error_dtr, 3)))
error_rfr = standard_error(y_pred_rfr.tolist(), y_test.tolist())
print("standard error pour la random forest regression est égal à : " + str(round(error_rfr, 3)))

VISUALISATION DES VALEURS QUI IMPACTENT LES RESULTATS

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
feat_importances = pd.DataFrame(cl3.feature_importances_, index = feats.columns, columns = ['Importance'])
feat_importances.sort_values(by= 'Importance', ascending = False, inplace = True )
feat_importances.plot(kind ='bar', figsize = (8,6))